In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [3]:
driver = webdriver.Chrome('./chromedriver')

오른쪽 위 url
https://map.kakao.com/?map_type=TYPE_MAP&q=%EC%A0%9C%EC%A3%BC+%EC%B9%B4%ED%8E%98&urlX=500000&urlY=15000&urlLevel=7
    
왼쪽 아래 url
https://map.kakao.com/?map_type=TYPE_MAP&q=%EC%A0%9C%EC%A3%BC+%EC%B9%B4%ED%8E%98&urlX=300000&urlY=-80000&urlLevel=7
    
X범위 : 300000 ~ 500000 / 200000 => 20 번 10000씩 마이너스
Y범위 : -80000 ~ 15000 / 95000 => 10 번 9500씩 마이너스

In [3]:
# time.sleep(2)
# driver.find_element_by_xpath('/html/body/div[10]/div/div/div/span').click()
# time.sleep(1)
# driver.find_element_by_xpath('//*[@id="info.search.place.more"]').click()

In [4]:
def next_page(): #다음 페이지 넘기는 함수
    driver.find_element_by_xpath('//*[@id="info.search.page.next"]').click()
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser') # 가져온 html 파일을 html parser를 통해서 정리

In [5]:
def make_links(list_): #아이디 뽑아오는 함수 
    try:
        for i in range(5):
            driver.find_element_by_xpath(f'//*[@id="info.search.page.no{i+1}"]').click()

            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
#             print('try1')
            for link in soup.find_all("a",attrs = {"data-id" : "moreview"}):
                list_.append(link['href'])
#             print('try2')
        next_page()
    
        return list_
    
    except: #구간을 쪼개고 링크를 빼오고 페이지 넘기고 아이디어 수집하게 반복 
        try:
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            for link in soup.find_all("a",attrs = {"data-id" : "moreview"}):
                list_.append(link['href'])
#             print('try-try')
        except:
#             print('false')
            return list_

In [6]:
cafe_list = []

In [7]:
# 시작 위치 설정
params_X = 500000
params_Y = 15000

In [8]:
# 시작 url
url = f'https://map.kakao.com/?map_type=TYPE_MAP&q=%EC%A0%9C%EC%A3%BC+%EC%B9%B4%ED%8E%98&urlX={params_X}&urlY={params_Y}&urlLevel=6'

In [9]:
driver.get(url)

In [10]:
time.sleep(1)
driver.find_element_by_xpath('/html/body/div[10]/div/div/div/span').click()

### 예외처리 (try, except)
에러가 발생할 가능성이 있는 코드를 try에 입력하고, except에 에러 종류와 해당 에러가 발생했을때 실행시킬 코드를 입력한다.  
이 방법을 이용하면 error가 발생해도 프로그램이 멈추지 않고 별도 처리를 할 수 있다.

In [11]:
count = 1

for _ in range(41):
    for _ in range(21):
        # url 열기
        time.sleep(1)
        driver.get(url)
        # 지도 내 장소검색 클릭
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="search.keyword.bounds"]').click()

        try:
            # 장소 더보기 클릭
            time.sleep(2)
            driver.find_element_by_xpath('//*[@id="info.search.place.more"]').click()
            # 페이지 선택 + 링크 저장
            for i in range(7):    
                make_links(cafe_list)
                params_Y -= 4750
                url = f'https://map.kakao.com/?map_type=TYPE_MAP&q=%EC%A0%9C%EC%A3%BC+%EC%B9%B4%ED%8E%98&urlX={params_X}&urlY={params_Y}&urlLevel=6'
        except:
            make_links(cafe_list)
            params_Y -= 4750
            url = f'https://map.kakao.com/?map_type=TYPE_MAP&q=%EC%A0%9C%EC%A3%BC+%EC%B9%B4%ED%8E%98&urlX={params_X}&urlY={params_Y}&urlLevel=6'
            
    params_Y = 15000
    params_X -= 5000
    url = f'https://map.kakao.com/?map_type=TYPE_MAP&q=%EC%A0%9C%EC%A3%BC+%EC%B9%B4%ED%8E%98&urlX={params_X}&urlY={params_Y}&urlLevel=6'

In [12]:
cafe_list

['https://place.map.kakao.com/495424696',
 'https://place.map.kakao.com/1082954673',
 'https://place.map.kakao.com/26894210',
 'https://place.map.kakao.com/26891735',
 'https://place.map.kakao.com/1868305338',
 'https://place.map.kakao.com/495424696',
 'https://place.map.kakao.com/489755369',
 'https://place.map.kakao.com/2017506919',
 'https://place.map.kakao.com/184819082',
 'https://place.map.kakao.com/153091550',
 'https://place.map.kakao.com/1082954673',
 'https://place.map.kakao.com/998666015',
 'https://place.map.kakao.com/26894210',
 'https://place.map.kakao.com/26891735',
 'https://place.map.kakao.com/1868305338',
 'https://place.map.kakao.com/495424696',
 'https://place.map.kakao.com/489755369',
 'https://place.map.kakao.com/2017506919',
 'https://place.map.kakao.com/184819082',
 'https://place.map.kakao.com/153091550',
 'https://place.map.kakao.com/1082954673',
 'https://place.map.kakao.com/998666015',
 'https://place.map.kakao.com/26894210',
 'https://place.map.kakao.com/26

In [13]:
len(cafe_list)

22555

In [14]:
cafe_id = set(cafe_list)

In [15]:
len(cafe_id)

2208

In [16]:
pd.DataFrame(cafe_id)

,0
0,https://place.map.kakao.com/1105114080
1,https://place.map.kakao.com/21491895
2,https://place.map.kakao.com/1540027970
3,https://place.map.kakao.com/11327203
4,https://place.map.kakao.com/1735338702
5,https://place.map.kakao.com/771528594
6,https://place.map.kakao.com/114853921
7,https://place.map.kakao.com/677411887
8,https://place.map.kakao.com/335498272
9,https://place.map.kakao.com/2020231452


In [19]:
cafe = []
for i in range(len(cafe_list)):
    cafe.append(cafe_list[i][28:])

In [21]:
cafe = set(cafe)

In [23]:
len(cafe)

2208

In [24]:
pd.DataFrame(cafe).to_csv("cafe_id.csv")